In [1]:
%set_env GPU=1
%set_env KOPT=1

env: GPU=1


In [2]:
import numpy as np
from PIL import Image
from tinygrad.tensor import Tensor
from tinygrad.nn import Conv2d, BatchNorm2d, Linear, optim
from tinygrad.state import get_parameters
from extra.datasets import fetch_cifar
from extra.training import train, evaluate
Tensor.manual_seed(1337)

In [3]:
class BasicBlock:
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        self.conv1 = Conv2d(inplanes, planes, 3, stride=stride, padding=1, bias=False)
        self.bn1 = BatchNorm2d(planes)
        self.conv2 = Conv2d(planes, planes, 3, stride=1, padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes)
        self.downsample = downsample
    
    def __call__(self, x):
        out = self.bn1(self.conv1(x)).relu()
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            x = x.sequential(self.downsample)
        out = (out + x).relu()
        return out

In [4]:
class ResNet:
    def __init__(self):
        self.inplanes = 64
        self.conv1 = Conv2d(3, 64, 7, stride=2, padding=3, bias=False)
        self.bn1 = BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 3, stride=1)
        self.layer2 = self._make_layer(128, 4, stride=2)
        self.layer3 = self._make_layer(256, 6, stride=2)
        self.layer4 = self._make_layer(512, 3, stride=2)
        self.fc = Linear(512, 10)
    
    def _make_layer(self, planes, blocks, stride=1):
        downsample = []
        if stride != 1 or self.inplanes != planes:
            downsample = [
                Conv2d(self.inplanes, planes, 1, stride=stride, bias=False),
                BatchNorm2d(planes),
            ]
        layers = [BasicBlock(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.inplanes, planes))
        return layers
    
    def __call__(self, x):
        x = self.bn1(self.conv1(x)).relu()
        x = x.pad2d((1, 1, 1, 1))
        x = x.max_pool2d((3, 3), stride=2)
        x = x.sequential(self.layer1)
        x = x.sequential(self.layer2)
        x = x.sequential(self.layer3)
        x = x.sequential(self.layer4)
        x = x.avg_pool2d((7, 7), stride=1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x).log_softmax()
        return x

In [5]:
def transform(x):
    x = [[Image.fromarray(z).resize((224, 224)) for z in y] for y in x]
    x = np.stack([np.stack([np.asarray(z) for z in y], axis=0) for y in x], axis=0)
    x = x.reshape(-1, 3, 224, 224)
    return x

In [6]:
(X_train, Y_train), (X_test, Y_test) = fetch_cifar(), fetch_cifar(train=False)
model = ResNet()
optimizer = optim.SGD(get_parameters(model), lr=0.005, weight_decay=0.0005, momentum=0.9)
train(model, X_train, Y_train, optimizer, 50000, BS=16, transform=transform)
evaluate(model, X_test, Y_test, transform=transform)

loss 3.24 accuracy 0.06:   0%|          | 20/50000 [00:25<13:45:02,  1.01it/s]